In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
import torch.distributions as td

import numpy as np
from numpy.random import normal, randint, choice

import scipy as sp
from scipy.special import softmax

import matplotlib.pyplot as plt
from collections import defaultdict

In [5]:
def loss_fn(V_pred, V_gt):
    """
    Arguments
    ---------
    G : torch.tensor[1]
    V : torch.tensor[1]

    Returns
    -------
    MSE : torch.tensor[1]
    """
    MSE = ((V_pred - V_gt) ** 2).mean()
    return MSE

In [20]:
from torch.utils.tensorboard import SummaryWriter

In [14]:
world = MDPWorld(10,10)
agent = DoubleDQN()
agent.place(world)
agent.learn() # gute nummer: 3000-100

In [15]:
world.V(pi = agent.pi.probs).sum()
world.V_opt.sum()

tensor(143.2325)

In [16]:
world.V_opt.sum()

tensor(172.3668)